In [1]:
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain.document_loaders.web_base import WebBaseLoader
import json


def get_websites_by_wikipedia_search(inputs):
    w = WikipediaAPIWrapper()
    query = inputs["query"]
    return w.run(query)


def get_websites_by_duckduckgo_search(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    query = inputs["query"]
    return ddg.run(query)


def get_document_text(inputs):
    url = inputs["url"]
    loader = WebBaseLoader([url])
    docs = loader.load()
    return docs[0].page_content


functions_map = {
    "get_websites_by_wikipedia_search": get_websites_by_wikipedia_search,
    "get_websites_by_duckduckgo_search": get_websites_by_duckduckgo_search,
    "get_document_text": get_document_text,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_websites_by_wikipedia_search",
            "description": "Use this tool to find the websites for the given query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query you will search for. Example query: Research about the XZ backdoor",
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_websites_by_duckduckgo_search",
            "description": "Use this tool to find the websites for the given query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query you will search for. Example query: Research about the XZ backdoor",
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_document_text",
            "description": "Use this tool to load the website for the given url.",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "The url you will load. Example url: https://en.wikipedia.org/wiki/Backdoor_(computing)",
                    }
                },
                "required": ["url"],
            },
        },
    },
]

In [3]:
import openai as client

assistant = client.beta.assistants.create(
    name="Investor Assistant",
    instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
    model="gpt-4-1106-preview",
    tools=functions,
)
assistant_id = assistant.id
assistant_id
# assistant_id = "asst_clhG8awYQ8VSrYO86Cest8U7"

'asst_pOGzLuG3Wo3qV7GK3fhJ3N9U'

In [5]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Research about the XZ backdoor",
        }
    ]
)
thread

Thread(id='thread_DkcukOj9IVsm8Tp0Um1dim4G', created_at=1713109012, metadata={}, object='thread')

In [6]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_gpW36ospbshfQ9aQ3Q47k8gb', assistant_id='asst_pOGzLuG3Wo3qV7GK3fhJ3N9U', cancelled_at=None, completed_at=None, created_at=1713109019, expires_at=1713109619, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_DkcukOj9IVsm8Tp0Um1dim4G', tools=[FunctionTool(function=FunctionDefinition(name='get_websites_by_wikipedia_search', description='Use this tool to find the websites for the given query.', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The query you will search for. Example query: Research about the XZ backdoor'}}, 'required': ['query']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_websites_by_duckduckgo_search', description='Use this tool to f

In [7]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outpus,
    )

In [8]:
get_tool_outputs(run.id, thread.id)

Calling function: get_websites_by_wikipedia_search with arg {"query":"XZ backdoor"}


[{'output': 'Page: XZ Utils backdoor\nSummary: On 29 March 2024, software developer Andres Freund reported that he had found a maliciously introduced backdoor in the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 released in February 2024.While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions.The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.\n\nPage: XZ Utils\nSummary: XZ Utils (previously LZMA Utils) is a set of free software command-line lossless data compressors, including the programs lzma and xz, for Unix-like operating systems and, from version 

In [9]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_websites_by_wikipedia_search with arg {"query":"XZ backdoor"}


Run(id='run_gpW36ospbshfQ9aQ3Q47k8gb', assistant_id='asst_pOGzLuG3Wo3qV7GK3fhJ3N9U', cancelled_at=None, completed_at=None, created_at=1713109019, expires_at=1713109619, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1713109019, status='queued', thread_id='thread_DkcukOj9IVsm8Tp0Um1dim4G', tools=[FunctionTool(function=FunctionDefinition(name='get_websites_by_wikipedia_search', description='Use this tool to find the websites for the given query.', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The query you will search for. Example query: Research about the XZ backdoor'}}, 'required': ['query']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_websites_by_duckduckgo_search', description='Use this too

In [19]:
get_run(run.id, thread.id).status

'completed'

In [20]:
get_messages(thread.id)

user: Research about the XZ backdoor
assistant: The XZ backdoor refers to a malicious backdoor discovered in the Linux utility xz, specifically within the liblzma library in versions 5.6.0 and 5.6.1 released in February 2024. The issue was reported by software developer Andres Freund on March 29, 2024. Despite xz being commonly present in most Linux distributions, at the time of the discovery, the backdoored versions had not been widely deployed to production systems. They were found in development versions of major distributions. This backdoor granted attackers with a specific Ed448 private key the capability to execute code remotely on affected Linux systems. It has been assigned the Common Vulnerabilities and Exposures (CVE) identifier CVE-2024-3094 and has received the highest possible score of 10.0 on the CVSS (Common Vulnerability Scoring System) scale, indicating a critical level of severity.

For context, XZ Utils is a set of free software command-line lossless data compressors

In [35]:
send_message(thread.id, "Now I want to know if Cloudflare is a good buy.")

Message(id='msg_KnyMu2vHcwsC7Jagjt5JgzOz', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Now I want to know if Cloudflare is a good buy.'), type='text')], created_at=1713103076, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_MErcuOfRNR0Ig1JjuPqQFSNp')